# COMP28512 Laboratory Task3: Bit-error Control
### Wenchang Liu 2019/03/11

## Part 3.1 Effect of increasing transmit poweron speech quality 

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
% matplotlib inline
from scipy.io import wavfile
from scipy.special import erfc
from comp28512_utils import Audio, audio_from_file, get_pesq_scores
from comp28512_utils import numpy_array_to_bit_array, bit_array_to_numpy_array, insert_bit_errors

# read speech from wavfile
(Fs, speech) = wavfile.read("NarrobandSpeech8k.wav")
print "Sampling frequency Fs as read from wav-file: ",Fs," Hz"
print("Narrow band speech without bit-errors:")
Audio(speech, rate=Fs)

fig, axs = plt.subplots(1)
x = np.arange(0.1, 20)
xe = 10*np.log10(x)
y_bep = 0.5*erfc(x**0.5)
axs.plot(xe, y_bep)
axs.set_yscale("log")
axs.grid(True, which="both")
axs.set_xlabel("x: Eb over N0(dB)") 
axs.set_ylabel("y: Bit-error probability")
axs.set_title("Bep against Eb/N0(Waterfall graph for msk)")  

# power(w), there is a 50dB(10^-5) power loss, N0 = 10*10^-12 w/Hz
def trans_power(power, speech):
    Eb = power/128000/(10**5)
    N0 = 10*10**(-12)
#     print 10*np.log10(Eb/N0)
    bep = 0.5*erfc((Eb/N0)**0.5)
#     bep = float(format(bep, ".0e"))
    print "bit-error probability for power "+str(power)+"w, 50db power loss, N0=10*10^-12w/Hz is: ", bep
    speech_with_errors = numpy_array_to_bit_array(speech)
    speech_with_errors = insert_bit_errors(speech_with_errors, bep)
    speech_with_errors = bit_array_to_numpy_array(speech_with_errors, np.int16)
    speech_name = "narroband_"+str(power)+"w.wav"
    ! rm $speech_name
    wavfile.write(speech_name, Fs, speech_with_errors)
    audio_from_file(speech_name)
    # score PESQ
    ! rm pesq_results.txt
    # Running PESQMain (in working directory with ‘chmod a+x’ set ) to obtain PESQ-MOS score:
    ! ./linux_pesqmain +8000 NarrobandSpeech8k.wav $speech_name > /dev/null
    # comp28512_utils must be in working directory
    pesq_results = get_pesq_scores()
    score = pesq_results["NarrobandSpeech8k.wav"] [speech_name] 
    print "PESQMain score for NarrobandSpeech8k.wav against "+speech_name+" = ", score
    print
    
    
trans_power(1.0, speech)
trans_power(1.35, speech)

### Part 3.1 Q&A:
1. Explain how the appropriate bit-error probability is calculated in Python for a transmission power of 1 watt.
   * Assume using 1 watt of power to send speech at 128k(Fs:8000 x NB:16) bits per second, and 50db(factor 10^-5) loss over channel.
   * Eb = power/128k/10^5 = 1.0/128k/10^5
   * Channel bandwidth: 30000Hz, Noise power is 300 x 10^-9 watts
   * The power spectral density, N0 = (300^10-9)/30000  = 10 x 10^-12 watts/Hz 
   * Bit-error probability bep = 0.5 x erfc((Eb/N0)^0.5)
2. Show how the required graph of bit-error probability against Eb/No (in dB) is produced in Python.
   * First, set a numpy array for "Eb/N0"s
   * Then, calculate "Eb/N0"(convert to dB: 10 x np.log10(x)) as x and Bit-error probability(0.5 x erfc(x^0.5)) as y
   * Finally, set y axis scale as "log" and plot the graph
3. What is the bit-error probability when the transmission power is 1 watt?
   * 3.86 x 10^-5
4. Explain how the evenly distributed bit-errors are introduced.
   * First, convert the speech(numpy array) we read from file to bit array by using provided numpy_array_to_bit_array function
   * Then use provided function insert_bit_errors(generate random evenly distributed erros with np.random.uniform according to given bep, and to exclusive-or with speech bit array) to add certain bit-errors to the bit array according to our bep(bit-error probability)
   * Finally, convert the bit array of speech with bit-errors back to numpy array by using provided bit_array_to_numpy_array function
5. What is the new bit-error probability (for 1.35 watts) and how was this calculated?
   * Assume using 1.35 watt of power to send speech at 128k bits per second, and 50db(factor 10^-5) loss over channel.
   * Eb = power/128k/10^5 = 1.35/128k/10^5
   * Channel bandwidth: 30000Hz, Noise power is 300 x 10^-9 watts
   * The power spectral density, N0 = (300^10-9)/30000  = 10 x 10^-12 watts/Hz 
   * bit-error probability bep = 0.5 x erfc((Eb/N0)^0.5) = 2.19 x 10^-6
6. What is the PESQ score for the narrowband speech transmitted with 1.35 watts?
   * Around 4.113 (each time is different).
7. How is the talk time affected?
   * watt = joule/s, since our phone has limited amount of battery, the more power we use for transmission, the less talk time will be available.
   * For example, assume our phone has battery of 18000 joules,
   * For 1 watt, we will have 18000/1 = 18000s talk time;
   * For 1.35 watt, we will have 18000/1.35 = 13333.3s talk time.
8. Explain in about one sentence how battery life and bit-error rates are connected.
   * Since we got less bit-errors using more power for transmission, and more power result in worse battery life, therefore, the lower bit-error rates we get, the worse battery life we get.

## Part 3.2 Effect of increasing bit-error probability on narrow-band speech without FEC scheme

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
% matplotlib inline
from scipy.io import wavfile
from scipy.special import erfc
from comp28512_utils import Audio, audio_from_file, get_pesq_scores
from comp28512_utils import numpy_array_to_bit_array, bit_array_to_numpy_array, insert_bit_errors

# read speech from wavfile
(Fs, speech) = wavfile.read("NarrobandSpeech8k.wav")
print "Sampling frequency Fs as read from wav-file: ",Fs," Hz"
print("Narrow band speech without bit-errors:")
Audio(speech, rate=Fs)


def trans_bep(bep, speech):
    print "speech with bit-error probability: ", bep
    speech_with_errors = numpy_array_to_bit_array(speech)
    speech_with_errors = insert_bit_errors(speech_with_errors, bep)
    speech_with_errors = bit_array_to_numpy_array(speech_with_errors, np.int16)
    speech_name = "narroband_bep"+".wav"
    wavfile.write(speech_name, Fs, speech_with_errors)
    audio_from_file(speech_name)
    # score PESQ
    ! rm pesq_results.txt
    # Running PESQMain (in working directory with ‘chmod a+x’ set ) to obtain PESQ-MOS score:
    ! ./linux_pesqmain +8000 NarrobandSpeech8k.wav $speech_name > /dev/null
    # comp28512_utils must be in working directory
    pesq_results = get_pesq_scores()
    score = pesq_results["NarrobandSpeech8k.wav"] [speech_name] 
    print "PESQMain score for NarrobandSpeech8k.wav against "+speech_name+" = ", score
    print
    return score


fig, axs = plt.subplots(1)
x1 = np.logspace(-5, -2, num=13)
y1 = np.zeros(13)
for i in range (0, 13):
    y1[i] = trans_bep(x1[i], speech)
axs.plot(x1, y1)
axs.set_xscale("log")
axs.grid(True, which="both")
axs.set_xlabel("x: bit-error rates") 
axs.set_ylabel("y: PESQ score")
axs.set_title("PESQ score against bep")    
    

### Part 3.2 Q&A:
1. How did you decide how to choose increasing values of bit-error probability? bearing in mind that your graph should have a logarithmic horizontal scale.
   * By using numpy.logspace to create a numpy array start from 10^-5 to 10^-2 which increases its value in logarithmic scale.
2. Comment on the effect of the bit-errors on the sound and how this changes as the bit-error probability increases.
   * There are more noises (clicks like) as we increase the bit-error probability.
3. Do the PESQ scores agree with your perception of the sound?
   * Yes. Basically, with high PESQ score, the sound is better, however, I cannot distinguish small differences between scores well(e.g. 1.4 and 1.5).

## Part 3.3 Effect of increasing bit-error probability on narrow-band speech with a (3,1) repetition FEC scheme

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
% matplotlib inline
from scipy.io import wavfile
from scipy.special import erfc
from comp28512_utils import Audio, audio_from_file, get_pesq_scores
from comp28512_utils import numpy_array_to_bit_array, bit_array_to_numpy_array, insert_bit_errors
from collections import Counter

# read speech from wavfile
(Fs, speech) = wavfile.read("NarrobandSpeech8k.wav")
print "Sampling frequency Fs as read from wav-file: ",Fs," Hz"
print("Narrow band speech without bit-errors:")
Audio(speech, rate=Fs)

def bepFec(bep, speech):
    print "speech with bit-error probability: ", bep
    speech_bit = numpy_array_to_bit_array(speech)
    speech_v1 = insert_bit_errors(speech_bit, bep)
    speech_v2 = insert_bit_errors(speech_bit, bep)
    speech_v3 = insert_bit_errors(speech_bit, bep)
    speech_with_errors = np.zeros(len(speech_bit))
    for i in range(0, len(speech_bit)):
        speech_with_errors[i] = Counter([speech_v1[i], speech_v2[i], speech_v3[i]]).most_common(1)[0][0]
    speech_with_errors = bit_array_to_numpy_array(speech_with_errors, np.int16)
    speech_name = "Fec_speech"+".wav"
    wavfile.write(speech_name, Fs, speech_with_errors)
    audio_from_file(speech_name)
    # score PESQ
    ! rm pesq_results.txt
    # Running PESQMain (in working directory with ‘chmod a+x’ set ) to obtain PESQ-MOS score:
    ! ./linux_pesqmain +8000 NarrobandSpeech8k.wav $speech_name > /dev/null
    # comp28512_utils must be in working directory
    pesq_results = get_pesq_scores()
    score = pesq_results["NarrobandSpeech8k.wav"] [speech_name] 
    print "PESQMain score for NarrobandSpeech8k.wav against "+speech_name+" = ", score
    print
    return score


x2 = np.logspace(-5, -2, num=13)
y2 = np.zeros(13)
y3 = np.zeros(13)
for i in range (0, 13):
    y2[i] = bepFec(x2[i], speech)
    y3[i] = round((y2[i]-y1[i])/y1[i]*100)
fig, axs = plt.subplots(1)
axs.plot(x2, y2)
axs.plot(x1, y1)
axs.legend(("with FEC", "without FEC"), loc="upper right")
axs.set_xscale("log")
axs.grid(True, which="both")
axs.set_xlabel("x: bit-error rates") 
axs.set_ylabel("y: PESQ score")
axs.set_title("PESQ score against bep")    

fig, axs = plt.subplots(1)
axs.plot(x2, y3)
axs.set_xscale("log")
axs.grid(True, which="both")
axs.set_xlabel("x: bit-error rates") 
axs.set_ylabel("y: PESQ score improve rate(%)")
axs.set_title("PESQ improve rate against bep")    

### Part 3.3 Q&A:
1. Demonstrate and comment on any improvement in the narrow-band speech quality that is obtained at the expense of the 3-fold increase in the bit-rate. How would you summarise the degree of improvement?
   * The improvement depends on the bit-error probability, if bep is too large or too small, then the improvement is not that significant, but if the bep is moderate, then we can acquire a great improvement up to over 200%.
2. What is the effect of the 3-fold increase in bit-rate on the transmit power and on the energy required to transmit the speech segment?
   * In part 3.3, we use the same conditons as part 3.2, so the transmit power will remain the same, and it means that we will need 3 times power for transmission.
3. 
   * (1) If we reduce the energy per bit by a factor of 3, the transmit power becomes what it was before?
     * 1/3
   * (2) Consider the example in Section 2.5 where Eb/N0 was 8.93 dB. It would now become 8.93-10 x log10(3) = 8.93-4.8 dB = 4.3dB. From the msk waterfall graph above, the beP now becomes about 0.01. How well does the (3,1) rep coder work at beP=0.01?
     * Small improvement, but not that good, not significant improvement.
4. What other method could be used to reduce the energy required to transmit the speech sentence without reducing the energy per bit? Think about previous Tasks.
   * Reduce sampling frequency or reduce the number of bits.
   * Eb = power/(FsxNB)/10^5 (10^5 is power loss). If we can reduce NB like previous Tasks or reduce sampling frequency, we can reduce power(reduce the energy required) and keep the energy per bit to still be the same.

## Part 3.4 Apply ARQ to a text message

In [ ]:
import numpy as np
from comp28512_utils import bytes_to_bit_array, bit_array_to_bytes, insert_bit_errors


# referenced from MS19myCRCdemo
def checkCRC(xa,gx) :  
    # Generates M-bit CRC as row-vector where length of gx is (M+1)
    # Generator polynomial of order M is represented by gx(0:M)
    # Message is bit-array xa(0:L)
    M = len(gx)-1
    # print "M = ", M
    # Does not append zeros to xa & assumes that this has been done already
    xsa = xa[0:M+1]
    for i in range (0 , len(xa) -(M)):
        if xsa[0] == gx[0]: 
            xsa = xsa ^ gx      
        xsa[0:M] = xsa[1:M+1]
        if ( i < len(xa)-M-1 ) :
            xsa[M] = xa[ i + M+1]
        # print(i, "xsa: ", xsa)
    check = xsa[0:M]    
    return check


# Generator polynomial is g(x) = x^8 + x^2 +x + 1
gx = [1,0,0,0,0,0,1,1,1];  # Generator polynomial as row-vector
text = "Smart-phones are mobile games consoles and mp3 players that you can also use for telephone calls."
def CRC_text(text, gx, bep):
    max_times = 9 # maximum allowed resend times
    count = 0 # already resend times
    equal_flag = 0 # if the check suceed
    while count < max_times and equal_flag == 0:
        equal_flag = 1
        text_bit = bytes_to_bit_array(text) # string -> bits 
        text_CRC = np.append(text_bit, [0,0,0,0,0,0,0,0]) # append zeros
        check = checkCRC(text_CRC, gx) # calculate check bit
        text_CRC = np.append(text_bit, check) # append new check bit and prepare for transmit
        # transmission
        text_received_CRC = insert_bit_errors(text_CRC, bep)
        check = text_received_CRC[len(text_received_CRC)-8:len(text_received_CRC)].copy() # received check bits
        text_received = bit_array_to_bytes(text_received_CRC[0:len(text_received_CRC)-8]) # received text
        text_received_CRC[len(text_received_CRC)-8:len(text_received_CRC)] = [0,0,0,0,0,0,0,0] # append zeros
        check_received = checkCRC(text_received_CRC, gx) # recalculate check bits and compare with received check bit

        # print check
        # print check_received
        for i in range(0, len(check)):
            if check[i] != check_received[i]:
                equal_flag = 0 # if check failed
                break

        print "bit-error probability = ", bep
        print "Now trying: ", count+1, "/", max_times
        if equal_flag == 1:
            print "CRC succeed"
        else:
            print "CRC failed"
        print "Original text:", text
        print "Received text:", text_received
        if count+1 == max_times:
            print "ARQ failed completely"
        print
        count += 1
    print
    

bep = np.logspace(-4, -1, num=13)
for i in range (0, 13):
    print "CRC test No.", i+1
    CRC_text(text, gx, bep[i])

### Part 3.4 Q&A:
1. If the CRC8 check succeeds, can we deduce that there are no bit-errors? 
   * No. Any combination of bit-errors that 'adds' any 'multiple' of G(x) will not be detected.
2. If the CRC8 check fails, can we deduce that there are some bit-errors? 
   * Yes.
3. Explain how your results demonstrate the effect of evenly distributed bit-errors and ARQ with various values of bit-error probability.
   * As the bit-error probability increases, there are more likely to be more evenly distributed bit-erros, so that ARQ will require more retransmissions, and when the bit-error probability is at a very large value(around 0.01), ARQ may fail completely.
4. For what values of bit-error probability did the ARQ process fail completely?
   * Each time the results can be different, but it is very likely that the ARQ process fail completely when bit-error probability is around 0.005623413251903491.

## Part 3.5 Applying a modified form of ARQ to a text message

In [ ]:
import numpy as np
from collections import Counter
from comp28512_utils import bytes_to_bit_array, bit_array_to_bytes, insert_bit_errors


# referenced from MS19myCRCdemo
def checkCRC(xa,gx) :  
    # Generates M-bit CRC as row-vector where length of gx is (M+1)
    # Generator polynomial of order M is represented by gx(0:M)
    # Message is bit-array xa(0:L)
    M = len(gx)-1
    # print "M = ", M
    # Does not append zeros to xa & assumes that this has been done already
    xsa = xa[0:M+1]
    for i in range (0 , len(xa) -(M)):
        if xsa[0] == gx[0]: 
            xsa = xsa ^ gx      
        xsa[0:M] = xsa[1:M+1]
        if ( i < len(xa)-M-1 ) :
            xsa[M] = xa[ i + M+1]
        # print(i, "xsa: ", xsa)
    check = xsa[0:M]    
    return check


# Generator polynomial is g(x) = x^8 + x^2 +x + 1
gx = [1,0,0,0,0,0,1,1,1];  # Generator polynomial as row-vector
text = "Smart-phones are mobile games consoles and mp3 players that you can also use for telephone calls."
def CRC_text(text, gx, bep):
    equal_flag = 1 # if the check suceed
    count = 0
    text_bit = bytes_to_bit_array(text) # string -> bits 
    text_CRC = np.append(text_bit, [0,0,0,0,0,0,0,0]) # append zeros
    check = checkCRC(text_CRC, gx) # calculate check bit
    text_CRC = np.append(text_bit, check) # append new check bit and prepare for transmit
    
    # transmission 1
    text_received_CRC = insert_bit_errors(text_CRC, bep)
    text_received_CRC1 = text_received_CRC.copy()
    check = text_received_CRC[len(text_received_CRC)-8:len(text_received_CRC)].copy() # received check bits
    text_received = bit_array_to_bytes(text_received_CRC[0:len(text_received_CRC)-8]) # received text
    text_received_CRC[len(text_received_CRC)-8:len(text_received_CRC)] = [0,0,0,0,0,0,0,0] # append zeros
    check_received = checkCRC(text_received_CRC, gx) # recalculate check bits and compare with received check bit

    for i in range(0, len(check)):
        if check[i] != check_received[i]:
            equal_flag = 0 # if check failed
            break

    print "bit-error probability = ", bep
    print "Now trying: transmission No.", count+1
    if equal_flag == 1:
        print "CRC succeed"
    else:
        print "CRC failed"
    print "Original text:", text
    print "Received text:", text_received
    print
    if equal_flag == 1:
        return
    count += 1
    
    # transmission 2
    equal_flag = 1
    text_received_CRC = insert_bit_errors(text_CRC, bep)
    text_received_CRC2 = text_received_CRC.copy()
    check = text_received_CRC[len(text_received_CRC)-8:len(text_received_CRC)].copy() # received check bits
    text_received = bit_array_to_bytes(text_received_CRC[0:len(text_received_CRC)-8]) # received text
    text_received_CRC[len(text_received_CRC)-8:len(text_received_CRC)] = [0,0,0,0,0,0,0,0] # append zeros
    check_received = checkCRC(text_received_CRC, gx) # recalculate check bits and compare with received check bit

    for i in range(0, len(check)):
        if check[i] != check_received[i]:
            equal_flag = 0 # if check failed
            break

    print "bit-error probability = ", bep
    print "Now trying: transmission No.", count+1
    if equal_flag == 1:
        print "CRC succeed"
    else:
        print "CRC failed"
    print "Original text:", text
    print "Received text:", text_received
    print
    if equal_flag == 1:
        return
    count += 1
    
    # transmission 3
    equal_flag = 1
    text_received_CRC = insert_bit_errors(text_CRC, bep)
    text_received_CRC3 = text_received_CRC.copy()
    check = text_received_CRC[len(text_received_CRC)-8:len(text_received_CRC)].copy() # received check bits
    text_received = bit_array_to_bytes(text_received_CRC[0:len(text_received_CRC)-8]) # received text
    text_received_CRC[len(text_received_CRC)-8:len(text_received_CRC)] = [0,0,0,0,0,0,0,0] # append zeros
    check_received = checkCRC(text_received_CRC, gx) # recalculate check bits and compare with received check bit

    for i in range(0, len(check)):
        if check[i] != check_received[i]:
            equal_flag = 0 # if check failed
            break

    print "bit-error probability = ", bep
    print "Now trying: transmission No.", count+1
    if equal_flag == 1:
        print "CRC succeed"
    else:
        print "CRC failed"
    print "Original text:", text
    print "Received text:", text_received
    print
    if equal_flag == 1:
        return
    else:
        # vote: text_received_CRC1,2,3
        equal_flag = 1
        print "Voting on the three failed transmissions..."
        vote_CRC = np.zeros(len(text_received_CRC1))
        for i in range(0, len(text_received_CRC1)):
            vote_CRC[i] = Counter([text_received_CRC1[i], text_received_CRC2[i], text_received_CRC3[i]])\
                          .most_common(1)[0][0]
        vote_CRC = np.int16(vote_CRC)
        # CRC check again
        check = vote_CRC[len(vote_CRC)-8:len(vote_CRC)].copy() # received check bits
        text_received = bit_array_to_bytes(vote_CRC[0:len(vote_CRC)-8]) # received text
        vote_CRC[len(vote_CRC)-8:len(vote_CRC)] = [0,0,0,0,0,0,0,0] # append zeros
        check_received = checkCRC(vote_CRC, gx) # recalculate check bits and compare with received check bit

        for i in range(0, len(check)):
            if check[i] != check_received[i]:
                equal_flag = 0 # if check failed
                break

        print "bit-error probability = ", bep
        print "Now checking voting result.."
        if equal_flag == 1:
            print "chase combining succeed"
        else:
            print "chase combining failed"
        print "Original text:", text
        print "Received text:", text_received
        print
        
bep = np.logspace(-4, -1, num=13)
for i in range (0, 13):
    print "CRC test No.", i+1
    CRC_text(text, gx, bep[i])

### Part 3.5 Q&A:
1. What could the ARQ mechanism above do if a numerical or string array is received correctly but bit-errors have occurred in the CRC8 check-bits?  How often might this occur?
   * It will fail CRC check and require retransmission, but it is very rare to happen.
2. State whether you consider ARQ as used in Part 3.4 an efficient method, and explain the reasons for your answer.
   * It is not very efficient because each time we are doing the same thing from scratch again and have the same probability of avoiding errors, and hoping we can be lucky enough. However, if we do it like Part 3.5, we can absorb the failing experience when we do the voting, which increase our probability of succeeding, and we save time to do less resending, we even get better results on high bit-error probability situations.
3. Explain your mechanism for combining failed retransmissions.
   * We firstly allow up to 3 times retransmissions, if we can success within 3 tries, the function stops. Otherwise, instead of retransmit again, we do as follows:
     * For each bit in the text bits array, we vote by selecting the most common bit according to the 3 failed bits array for our new bits array.
     * After we get the voting text bits array, we do the same CRC checking as we did after transmission previously, and check whether the voting text can pass CRC or not, we can also print out the new text and compare with the original text to see the chase combining performance. 
4. Demonstrate the effect of combining 3 failed transmissions. 
   * It works pretty well, even if bep is around 0.01 (3.4 ARQ definitely failed completey), our chase combining method still can get the correct results. Even for situations where failed completely, we can get less errors than part 3.4.
5. At what value of beP does the new ARQ process fail completely? Is there any improvement over what we got before?
   * Every time the results are different, however, it is very likely that ARQ fails if bep reaches 0.03.
   * The improvement is significant, sometime we can correct high bep(like around 0.01), and the average bep for new ARQ process to fail completely is higher than the previous one.
6. If combining failed transmissions turns out to be a good idea, why is it not used in practice, for example in wifi (you need to look this up).
   * Current wifi not using harq(chase combine and incremental redundancy), mainly because:
     * Concerns regarding combining retransmissions in the wifi(unlicensed band) which is (sometimes) highly interfered:
       * Impossible to know which transmission and retransmission should be combined.
       * Failure due to decoding error and collisions are not distinguished(we prefer not to combine collision failed transmissions).
       * Combining blindly different transmissions could result combining incorrect data from different         transmitters.
       * Incorrectly received data may not be even intended for receiver doing the combining
     * Overheads for extra works and implementation difficulties for using HARQ
   * Do think people are researching on incorporating HARQ into 802.11 standard.
   * Reference: 
     * doc.: IEEE 802.11-18/1955r0
     * doc.: IEEE 802.11-13/0314-00
